In [0]:
from IPython.display import HTML

In [2]:
!pip install gym-retro

In [3]:
!wget http://www.atarimania.com/roms/Roms.rar
!unrar x Roms.rar
!unzip ./Roms/ROMS.zip
%cd ROMS
!python -m retro.import .

--2019-01-03 11:53:14--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10823448 (10M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.2’

Roms.rar.2          100%[===================>]  10.32M   621KB/s    in 18s     

2019-01-03 11:53:31 (599 KB/s) - ‘Roms.rar.2’ saved [10823448/10823448]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar


Would you like to replace the existing file Roms/HC ROMS.zip
11729845 bytes, modified on 2018-12-24 16:58
with a new one
11729845 bytes, modified on 2018-12-24 16:58

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit N


Would you like to replace the existing file Roms/ROMS.zip
7826740 bytes, modified on 2018-12-24 16:58
with a new one
7826740 bytes, modified on 2018-12-24 16:58

[Y]es, [N]o, [A]ll, n[E]v

In [0]:
import tensorflow as tf
import numpy as np
import retro

from skimage import transform
from skimage.color import rgb2gray

import matplotlib.pyplot as plt

from collections import deque

import random

import warnings

warnings.filterwarnings('ignore')

In [5]:
env = retro.make(game='SpaceInvaders-Atari2600')

print('The size of our frame is: ', env.observation_space)
print('The action size is: ', env.action_space.n)

possible_actions = np.array(np.identity(env.action_space.n, dtype=int).tolist())

The size of our frame is:  Box(210, 160, 3)
The action size is:  8


In [0]:
def preprocess_frame(frame):
  gray = rgb2gray(frame)
  cropped_frame = gray[8:-12, 4:-12]
  normalized_frame = cropped_frame/255.0
  preprocessed_frame = transform.resize(normalized_frame, [110,84])
  return preprocessed_frame

In [0]:
stack_size = 4

stacked_frames = deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
  frame = preprocess_frame(state)
  
  if is_new_episode:
    stacked_frames = deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
    
    stacked_frames.append(frame)
    stacked_frames.append(frame)
    stacked_frames.append(frame)
    stacked_frames.append(frame)
    
    stacked_state = np.stack(stacked_frames, axis=2)
    
  else:
    stacked_frames.append(frame)
    
    stacked_state = np.stack(stacked_frames, axis=2)
    
  return stacked_state, stacked_frames
    

In [0]:
state_size = [110, 84, 4]

action_size = env.action_space.n
learning_rate = 0.00025           # alpha

total_episodes = 50
max_steps = 50000
batch_size = 64

explore_start = 1.0
explore_stop = 0.01
decay_rate = 0.00001

gamma = 0.9

pretrain_length = batch_size # no. of experiences stored in the memory when initialized for the first time
memory_size = 1000000 # no. of experiences the memory can keep

stack_size = 4

training = True # false to just see the trained agent
episode_render = False # true if you want to render the env

In [0]:
class DQNetwork:
  def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
    self.state_size = state_size
    self.action_size = action_size
    self.learning_rate = learning_rate
    
    with tf.variable_scope(name):
      
      self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
      self.actions_ = tf.placeholder(tf.float32, [None, self.action_size], name="actions_")
      
      # target_Q = R(s,a) + ymax Qhat(s', a')
      self.target_Q = tf.placeholder(tf.float32, [None], name="targets")
      
      self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                 filters = 32,
                                 kernel_size = [8,8],
                                 strides = [4,4],
                                 padding = "VALID",
                                 kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                 name = "conv1")
      
      self.conv1_out = tf.nn.elu(self.conv1, name="conv1_out")
      
      self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                   filters = 64,
                                   kernel_size=[4,4],
                                   strides = [2,2],
                                   padding = "VALID",
                                   kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                   name = "conv2")
      
      self.conv2_out = tf.nn.elu(self.conv2, name="conv2_out")
      
      self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                   filters = 64,
                                   kernel_size= [3,3],
                                   strides = [2,2],
                                   padding = "VALID",
                                   kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                   name = "conv3")
      self.conv3_out = tf.nn.elu(self.conv3, name="conv3_out")
      
      self.flatten = tf.contrib.layers.flatten(self.conv3_out)
      
      self.fc = tf.layers.dense(inputs = self.flatten,
                               units = 512,
                               activation = tf.nn.elu,
                               kernel_initializer = tf.contrib.layers.xavier_initializer(),
                               name = "fc1")
      
      self.output = tf.layers.dense(inputs = self.fc,
                                   kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                   units = self.action_size,
                                   activation = None)
      self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_))
      
      self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
      
      self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)

In [0]:
tf.reset_default_graph()

DQNetwork = DQNetwork(state_size, action_size, learning_rate)

In [0]:
class Memory():
  def __init__(self, max_size):
    self.buffer = deque(maxlen = max_size)
    
  def add(self, experience):
    self.buffer.append(experience)
    
  def sample(self, batch_size):
    buffer_size = len(self.buffer)
    index = np.random.choice(np.arange(buffer_size),
                            size = batch_size,
                            replace = False)
    return [self.buffer[i] for i in index]

In [0]:
memory = Memory(max_size = memory_size)
for i in range(pretrain_length):
  
  if i ==0:
    state = env.reset()
    
    state, stacked_frames = stack_frames(stacked_frames, state, True)
    
  choice = random.randint(1, len(possible_actions))-1
  action = possible_actions[choice]
  next_state, reward, done, _ = env.step(action)
  
  next_state, stacked_frames = stack_frames(stacked_frames, next_state, True)
  
  if done:
    next_state = np.zeros(state.shape)
    
    memory.add((state, action, reward, next_state, done))
    
    stae = env.reset()
    
    state, stacked_frames = stack_frames(stacked_frames, state, True)
    
  else:
    memory.add((state, action, reward, next_state, done))
    
    state = next_state

In [0]:
writer = tf.summary.FileWriter("/tensorboard/dqn/1")

tf.summary.scalar("Loss", DQNetwork.loss)

write_op = tf.summary.merge_all()

In [0]:
def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
  
  exp_exp_tradeoff = np.random.rand()
  
  explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
  
  if(explore_probability > exp_exp_tradeoff):
    choice = random.randint(1, len(possible_actions)) -1 
    action = possible_actions[choice]
    
  else:
    
    Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})
    
    choice = np.argmax(Qs)
    action = possible_actions[choice]
    
  return action, explore_probability

In [21]:
saver = tf.train.Saver()
rewards_list = []

if training == True:
  with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    decay_step = 0
    
    for episode in range(total_episodes):
      step = 0
      episode_rewards = []
      state = env.reset()
      
      state, stacked_frames = stack_frames(stacked_frames, state, True)
      
      while step < max_steps:
        step += 1
        
        decay_step += 1
        
        action, explore_probability = predict_action(explore_start, explore_stop, decay_rate,
                                                     decay_step, state, possible_actions)
        
        next_state, reward, done, _ = env.step(action)
        if episode_render:
          env.render()
          
        episode_rewards.append(reward)
          
        if done:
          next_state = np.zeros((110,84), dtype=np.int)
          
          next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
          
          step = max_steps
          
          total_reward = np.sum(episode_rewards)
          
          print('Episode: {}'.format(episode),
               'Total Reward: {}'.format(total_reward),
               'Explore P: {:.4f}'.format(explore_probability),
               'Training Loss {:.4f}'.format(loss))
          
          rewards_list.append((episode, total_reward))
          
          memory.add((state, action, reward, next_state, done))
        else:
          next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
          
          memory.add((state, action, reward, next_state, done))
          
          state = next_state
          
        batch = memory.sample(batch_size)
        states_mb = np.array([each[0] for each in batch], ndmin=3)
        actions_mb = np.array([each[1] for each in batch])
        rewards_mb = np.array([each[2] for each in batch])
        next_states_mb = np.array([each[3] for each in batch])
        dones_mb = np.array([each[4] for each in batch])
        
        target_Qs_batch = []
        
        Qs_next_state = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states_mb})
        
        for i in range (0, len(batch)):
          terminal = dones_mb[i]
          if terminal:
            target_Qs_batch.append(rewards_mb[i])
            
          else:
            target = rewards_mb[i] + gamma*np.max(Qs_next_state[i])
            target_Qs_batch.append(target)
            
        targets_mb = np.array([each for each in target_Qs_batch])
        
        loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
                          feed_dict={DQNetwork.inputs_: states_mb,
                                    DQNetwork.target_Q: targets_mb,
                                    DQNetwork.actions_: actions_mb})
        
        summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states_mb,
                                               DQNetwork.target_Q: targets_mb,
                                               DQNetwork.actions_: actions_mb})
        
        writer.add_summary(summary, episode)
        writer.flush()
        
      if episode % 5 == 0:
        save_path = saver.save(sess, "./models/model.ckpt")
        print("Model Saved")
          

Episode: 0 Total Reward: 120.0 Explore P: 0.9774 Training Loss 3.4796
Model Saved
Episode: 1 Total Reward: 35.0 Explore P: 0.9631 Training Loss 0.0034
Episode: 2 Total Reward: 255.0 Explore P: 0.9363 Training Loss 0.0045


KeyboardInterrupt: ignored

In [0]:
with tf.Session() as sess:
  total_test_rewards = []
  
  saver.restore(sess, "./models/model.ckpt")
  
  for episode in range(1):
    total_rewards = 0
    
    state = env.reset()
    state, stacked_frames = stack_frames(stacked_frames, state, True)
    
    print("******************************************************")
    print("EPISODE ", episode)
    while True:
      
      state = state.reshape((1, *state_size))
      
      Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state})
      
      choice = np.argmax(Qs)
      action = possible_actions[choice]
      
      next_state, reward, done, _ = env.step(action)
      env.render()
      
      total_rewards += reward
      
      if done:
        print("Score", total_rewards)
        total_test_rewards.append(total_rewards)
        break
        
      next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
      state = next_state
      
  env.close()

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
******************************************************
EPISODE  0
